In [1]:
import numpy as np
import cv2
from tkinter import Tk, filedialog, Button, Canvas, Frame, Label, StringVar, Scale, Text
from PIL import Image, ImageTk
import time

In [2]:
def pad_image(image, kernel, pad_type):
    height, width = kernel.shape
    pad_height = height // 2
    pad_width = width // 2
    
    if pad_type == 'clip':
        # Zero-padding
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant')
    elif pad_type == 'wrap':
        # Wrap around
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='wrap')
    elif pad_type == 'copy_edge':
        # Copy edge
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='edge')
    elif pad_type == 'reflect':
        # Reflect across edge
        padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='reflect')
    else:
        raise ValueError("Invalid padding type.")
    
    return padded_image

In [3]:
# My own conv2 from project2
def my_conv2(image, kernel, pad_type):
    # Ensure the kernel is 2D
    assert len(kernel.shape) == 2, "Kernel must be 2D."

    # Add a dummy channel dimension for grayscale image to simulate RGB image
    if len(image.shape) == 2:
        image = image[:, :, np.newaxis]

    # Acquire the dimensions of the image and kernel
    height, width, channels = image.shape
    k_height, k_width = kernel.shape

    # Pad the image based on the padding type
    padded_image = pad_image(image, kernel, pad_type)

    # Initialize an output array
    output = np.zeros((height, width, channels))

    # Perform the convolution
    for c in range(channels):
        for i in range(height):
            for j in range(width):
                region = padded_image[i:i + k_height, j:j + k_width, c]
                output[i, j, c] = np.sum(region * kernel)

    # Remove the dummy channel dimension for grayscale image
    if output.shape[2] == 1:
        output = output[:, :, 0]

    return output

In [4]:
# Optimized conv2 function using matrix slicing
def conv2(image, kernel, pad_type):
    """
    Perform a 2D convolution using matrix slicing.
    """
    # Ensure the kernel is 2D
    assert len(kernel.shape) == 2, "Kernel must be 2D."

    # Add a dummy channel dimension for grayscale image to simulate RGB image
    if len(image.shape) == 2:
        image = image[:, :, np.newaxis]

    # Acquire dimensions
    height, width, channels = image.shape
    k_height, k_width = kernel.shape

    # Pad the image based on the padding type
    padded_image = pad_image(image, kernel, pad_type)

    # Initialize the output array
    output = np.zeros((height, width, channels))

    # Flip the kernel for convolution
    flipped_kernel = kernel[::-1, ::-1]

    # Perform convolution channel by channel
    for c in range(channels):
        # Extract sliding windows for the current channel
        windows = np.lib.stride_tricks.sliding_window_view(
            padded_image[:, :, c], (k_height, k_width)
        )
        # Perform the convolution by reshaping windows and using matrix multiplication
        output[:, :, c] = np.tensordot(windows, flipped_kernel, axes=((2, 3), (0, 1)))

    # Remove the dummy channel dimension for grayscale image
    if output.shape[2] == 1:
        output = output[:, :, 0]

    return output

In [5]:
# Downsampling function
def downsample(image):
    return image[::2, ::2]

# Upsampling function
def upsample(image, target_size):
    if len(image.shape) == 3:  # RGB image
        upsampled = np.zeros((target_size[0], target_size[1], image.shape[2]), dtype=image.dtype)
        upsampled[::2, ::2, :] = image
    else:  # Grayscale image
        upsampled = np.zeros(target_size, dtype=image.dtype)
        upsampled[::2, ::2] = image

    # Smooth the upsampled image to reduce artifacts
    upsampled = cv2.GaussianBlur(upsampled, (5, 5), 0)

    return upsampled

In [6]:
# Compute Gaussian and Laplacian pyramids
def ComputePyr(input_image, num_layers):
    max_layers = int(np.floor(np.log2(min(input_image.shape[:2])))) - 1
    num_layers = min(num_layers, max_layers)

    # Gaussian kernel
    kernel = cv2.getGaussianKernel(5, 1)
    kernel = np.outer(kernel, kernel)

    # Initialize pyramids
    gPyr = [input_image.astype(np.float32)]
    lPyr = []

    for i in range(num_layers):
        # Smooth the image using custom conv2
        smoothed = conv2(gPyr[-1], kernel, 'wrap')

        # Downsample the smoothed image
        downsampled = downsample(smoothed)
        gPyr.append(downsampled)

        # Compute the Laplacian
        target_size = (gPyr[-2].shape[0], gPyr[-2].shape[1])
        upsampled = upsample(downsampled, target_size)
        laplacian = gPyr[-2] - upsampled
        lPyr.append(laplacian)

    lPyr.append(gPyr[-1])  # Add the smallest Gaussian layer as the last Laplacian layer
    return gPyr, lPyr

In [7]:
# Blend clipped image and target image according to mask
def blend_with_pyramids(target_image, clipped_image, mask):
    # Resize to match target image
    clipped_image = cv2.resize(clipped_image, (target_image.shape[1], target_image.shape[0]))
    mask = cv2.resize(mask, (target_image.shape[1], target_image.shape[0]))

    # Normalize and apply blur on mask
    mask_normalized = mask / 255.0
    smoothed_mask = cv2.GaussianBlur(mask_normalized, (35, 35), 0)  

    # Match the intensity of clipped image to target image
    def match_intensity(source, target, mask):
        source_mean = np.sum(source * mask[..., None], axis=(0, 1)) / np.sum(mask)
        target_mean = np.sum(target * mask[..., None], axis=(0, 1)) / np.sum(mask)
        scale = target_mean / (source_mean + 1e-5)
        adjusted = np.clip(source * scale, 0, 255).astype(np.uint8)
        return adjusted

    clipped_adjusted = match_intensity(clipped_image, target_image, mask_normalized)

    # Set number of level of pyramids and compute
    levels = 6
    gPyr_target, lPyr_target = ComputePyr(target_image, levels)
    gPyr_clipped, lPyr_clipped = ComputePyr(clipped_adjusted, levels)
    gPyr_mask, lPyr_mask = ComputePyr(mask_normalized, levels)

    # Start blending
    lPyr_blended = []
    for l_target, l_clipped, g_mask in zip(lPyr_target, lPyr_clipped, gPyr_mask):
        blended = l_target * (1 - g_mask[..., None]) + l_clipped * g_mask[..., None]
        lPyr_blended.append(blended)

    blended_image = lPyr_blended[-1]
    for i in range(len(lPyr_blended) - 2, -1, -1):
        blended_image = upsample(blended_image, lPyr_blended[i].shape[:2])
        blended_image = cv2.add(blended_image, lPyr_blended[i])

    # Apply smoothed/blurred mask on the final output
    final_result = target_image * (1 - smoothed_mask[..., None]) + blended_image * smoothed_mask[..., None]
    return np.clip(final_result, 0, 255).astype(np.uint8)


In [8]:
class Mask_Blending_GUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Mask Blending GUI")

        # Image and mask data
        self.target_image = None
        self.source_image = None
        self.black_background = None
        self.source_background = None
        self.temp_target = None
        self.resized_target = None
        self.temp_source = None
        self.mask = None
        self.output_image = None

        # Source placement variables
        self.x, self.y = 50, 50  # Initial position of the source
        self.dragging = False
        self.drawing = False
        self.mode = StringVar(value="drag")  # Default mode is drag
        self.mask_type = StringVar(value="rectangle")  # Default mask type is rectangle

        # Layout: Canvases with Labels
        canvas_frame = Frame(self.root)
        canvas_frame.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

        # Target Image
        Label(canvas_frame, text="Target Image").grid(row=0, column=0)
        self.target_canvas = Canvas(canvas_frame, width=500, height=650, bg="lightgray")
        self.target_canvas.grid(row=1, column=0)

        # Source Image
        Label(canvas_frame, text="Source Image").grid(row=0, column=1)
        self.background_canvas = Canvas(canvas_frame, width=500, height=650, bg="lightgray")
        self.background_canvas.grid(row=1, column=1)
        self.background_canvas.bind("<Button-1>", self.mouse_action_start)
        self.background_canvas.bind("<B1-Motion>", self.mouse_action)
        self.background_canvas.bind("<ButtonRelease-1>", self.mouse_action_stop)

        # Output Image
        Label(canvas_frame, text="Output Image").grid(row=0, column=2)
        self.output_canvas = Canvas(canvas_frame, width=500, height=650, bg="lightgray")
        self.output_canvas.grid(row=1, column=2)

        # Layout: Button Groups
        control_frame = Frame(self.root)
        control_frame.grid(row=1, column=0, sticky="w", padx=10, pady=10)

        self.add_button_groups(control_frame)

    def add_button_groups(self, control_frame):
        # Section: Load and Save
        load_save_frame = Frame(control_frame, relief="groove", borderwidth=2, padx=10, pady=10)
        load_save_frame.grid(row=0, column=0, padx=10, pady=5, sticky="w")
        Label(load_save_frame, text="Load/Save", font=("Arial", 10, "bold")).pack(anchor="w")
        Button(load_save_frame, text="Load Target Image", command=self.load_target_image, width=20).pack(anchor="w", pady=2)
        Button(load_save_frame, text="Load Source Image", command=self.load_source_image, width=20).pack(anchor="w", pady=2)
        Button(load_save_frame, text="Save Outputs", command=self.save_outputs, width=20).pack(anchor="w", pady=2)
        Button(load_save_frame, text="Save Clipped Image", command=self.save_clipped_image, width=20).pack(anchor="w", pady=2)
        Button(load_save_frame, text="Clear Images", command=self.clear_images, width=20).pack(anchor="w", pady=2)

        # Section: Resize Source Image
        resize_frame = Frame(control_frame, relief="groove", borderwidth=2, padx=5, pady=5)
        resize_frame.grid(row=0, column=1, padx=10, pady=5, sticky="w")
        Label(resize_frame, text="Resize Target Image", font=("Arial", 10, "bold")).pack(anchor="w")
        self.resize_target = Scale(resize_frame, from_=1, to=200, orient="horizontal", length=300, command=self.resize_target_image)
        self.resize_target.set(100)  # Default size is 100%
        self.resize_target.pack(pady=5)
        Button(resize_frame, text="Reset Size", command=self.reset_target_size, width=20).pack(pady=5)
        Label(resize_frame, text="Resize Source Image", font=("Arial", 10, "bold")).pack(anchor="w")
        self.resize_source = Scale(resize_frame, from_=1, to=200, orient="horizontal", length=300, command=self.resize_source_image)
        self.resize_source.set(100)  # Default size is 100%
        self.resize_source.pack(pady=5)
        Button(resize_frame, text="Reset Size", command=self.reset_source_size, width=20).pack(pady=5)
        
        # Section: Drag and Draw Mode Switch
        mode_frame = Frame(control_frame, relief="groove", borderwidth=2, padx=10, pady=10)
        mode_frame.grid(row=0, column=2, padx=10, pady=5, sticky="w")
        Label(mode_frame, text="Drag&Draw Mode Switch", font=("Arial", 10, "bold")).pack(anchor="w")
        Button(mode_frame, text="Drag Mode", command=lambda: self.set_mode("drag"), width=20).pack(anchor="w", pady=2)
        Button(mode_frame, text="Draw Mode", command=lambda: self.set_mode("draw"), width=20).pack(anchor="w", pady=2)
        
        # Section: Draw Mode and Masks
        mask_frame = Frame(control_frame, relief="groove", borderwidth=2, padx=10, pady=10)
        mask_frame.grid(row=0, column=3, padx=10, pady=5, sticky="w")
        Label(mask_frame, text="Select Mask", font=("Arial", 10, "bold")).pack(anchor="w")
        Button(mask_frame, text="Rectangle Mask", command=lambda: self.set_mask_type("rectangle"), width=20).pack(anchor="w", pady=2)
        Button(mask_frame, text="Ellipse Mask", command=lambda: self.set_mask_type("ellipse"), width=20).pack(anchor="w", pady=2)
        Button(mask_frame, text="Freeform Mask", command=lambda: self.set_mask_type("freeform"), width=20).pack(anchor="w", pady=2)
        Button(mask_frame, text="Eraser", command=lambda: self.set_mask_type("erase"), width=20).pack(anchor="w", pady=2)
        Button(mask_frame, text="Clear Mask", command=self.mask_cleared, width=20).pack(anchor="w", pady=2)
        Button(load_save_frame, text="Apply Pyramid Blending", command=self.apply_pyramid_blending, width=20).pack(pady=5)
        
        # Section: Log and Status Message
        status_frame = Frame(control_frame, relief="groove", borderwidth=2, padx=10, pady=10)
        status_frame.grid(row=0, column=4, columnspan=2, padx=10, pady=5, sticky="w")
        Label(status_frame, text="Log Messages", font=("Arial", 12)).pack(pady=5)
        self.status_text = Text(status_frame, wrap="word", height=10, state="disabled", font=("Arial", 10))
        self.status_text.pack(fill="both", expand=True)

    def set_mode(self, mode):
        self.mode.set(mode)
        self.log_message(f"Mode set to: {mode}.")
        
    def set_mask_type(self, mask_type):
        self.mask_type.set(mask_type)
        self.log_message(f"Mask type set to: {mask_type}.")
    
    def load_target_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.target_image = cv2.imread(file_path)
            self.target_image = cv2.cvtColor(self.target_image, cv2.COLOR_BGR2RGB)
            self.temp_target = self.target_image.copy()
            self.resize_target.set(100)
            
            if self.target_image is not None:
                self.output_image = self.target_image.copy()
                self.display_output_image()
            self.display_target_image()
            
            self.source_background = np.zeros_like(self.target_image)
            if self.source_image is not None:
                self.black_background = np.zeros_like(self.target_image)
                self.update_images()
            else:
                self.display_source_background()
            self.log_message("Target image loaded.")
            
    def load_source_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.source_image = cv2.imread(file_path)
            self.source_image = cv2.cvtColor(self.source_image, cv2.COLOR_BGR2RGB)
            self.temp_source = self.source_image.copy()
            self.resize_source.set(100)

            if self.target_image is None:
                self.log_message("Please load the target image first.")
                return

            # Generate a black background of the same size as the target
            if self.resized_target is not None:
                self.black_background = np.zeros_like(self.resized_target)
            else:
                self.black_background = np.zeros_like(self.target_image)

            # Initial placement of the source image
            self.x, self.y = 50, 50
            self.update_images()
            self.log_message("Source image loaded.")

    def resize_source_image(self, scale_value):
        if self.source_image is not None:
            # Convert the scale value to a percentage
            scale_percent = int(scale_value) / 100.0

            # Resize the width and height of source image
            original_h, original_w = self.source_image.shape[:2]
            new_w = max(1, int(original_w * scale_percent))  # Prevent zero or negative dimensions
            new_h = max(1, int(original_h * scale_percent))

            # Resize the source image
            self.temp_source = cv2.resize(self.source_image, (new_w, new_h), interpolation=cv2.INTER_AREA)
            if self.resized_target is not None:
                self.output_image = self.resized_target.copy()
                if self.mask is None or self.mask.shape[:2] != self.resized_target.shape[:2]:
                    self.mask = np.zeros(self.resized_target.shape[:2], dtype=np.uint8)
            else:
                self.output_image = self.target_image.copy()
                if self.mask is None or self.mask.shape[:2] != self.target_image.shape[:2]:
                    self.mask = np.zeros(self.target_image.shape[:2], dtype=np.uint8)

            # Update the display
            self.update_images()
            
    def resize_target_image(self, scale_value):
        if self.target_image is not None:
            # Convert the scale value to a percentage
            scale_percent = int(scale_value) / 100.0

            # Resize the width and height of target image
            original_h, original_w = self.target_image.shape[:2]
            new_w = max(1, int(original_w * scale_percent))  # Prevent zero or negative dimensions
            new_h = max(1, int(original_h * scale_percent))

            # Resize the target image
            self.resized_target = cv2.resize(self.target_image, (new_w, new_h), interpolation=cv2.INTER_AREA)
            self.output_image = self.resized_target.copy()
            if self.mask is None or self.mask.shape[:2] != self.resized_target.shape[:2]:
                self.mask = np.zeros(self.resized_target.shape[:2], dtype=np.uint8)
            self.black_background = np.zeros_like(self.resized_target)
            
            # Update the display
            if self.source_image is not None:
                self.update_images()
            else: # Only update the target-image-related images if source image is not loaded
                if self.resized_target is not None:
                    self.temp_target = self.resized_target.copy()
                else:
                    self.temp_target = self.target_image.copy()
                self.source_background = self.black_background.copy()
                self.display_source_background()
                self.display_target_image()
                self.display_output_image()
    
    def reset_source_size(self):
        if self.source_image is not None:
            # Reset slider to 100% (original size)
            self.resize_source.set(100)
            
            # Update the display
            self.update_images()
            self.log_message("Source image size reset.")
        else:
            self.log_message("Please load source image first.")
    
    def reset_target_size(self):
        if self.source_image is not None:
            # Reset slider to 100% (original size)
            self.resize_target.set(100)

            # Update the display
            self.update_images()
            self.log_message("Target image size reset.")
        else:
            self.log_message("Please load target image first.")
    
    def update_masked_source(self):
        if self.source_image is not None:
            # Ensure the mask matches the source image size
            if self.resized_target is not None:
                if self.mask is None or self.mask.shape[:2] != self.resized_target.shape[:2]:
                    self.mask = np.zeros(self.resized_target.shape[:2], dtype=np.uint8)
            elif self.mask is None or self.mask.shape[:2] != self.target_image.shape[:2]:
                self.mask = np.zeros(self.target_image.shape[:2], dtype=np.uint8)

            # Apply the resized mask to each channel of the source image
            if self.resized_target is not None:
                self.output_image = self.resized_target.copy()
                self.masked_source = np.zeros_like(self.resized_target)
            else:
                self.output_image = self.target_image.copy()  # Black background matches target size
                self.masked_source = np.zeros_like(self.target_image)
            for i in range(3):  # Iterate over RGB channels
                self.masked_source[:, :, i] = self.source_background[:, :, i] * (self.mask / 255)
                
            # Overlay the masked source on the target
            h, w = self.temp_source.shape[:2]
            x_start = max(self.x, 0)
            y_start = max(self.y, 0)
            x_end = min(self.x + w, self.source_background.shape[1])
            y_end = min(self.y + h, self.source_background.shape[0])
            self.output_image[y_start:y_end, x_start:x_end] = np.where(
                self.masked_source[y_start:y_end, x_start:x_end] != 0,
                self.masked_source[y_start:y_end, x_start:x_end],
                self.output_image[y_start:y_end, x_start:x_end]
            )
            
            self.update_images()
    
    def update_images(self):
        if self.black_background is not None and self.source_image is not None:
            # Update source background and target image
            if self.resized_target is not None:
                self.temp_target = self.resized_target.copy()
            else:
                self.temp_target = self.target_image.copy()
            self.source_background = self.black_background.copy()

            h, w = self.temp_source.shape[:2]
            x_start = max(self.x, 0)
            y_start = max(self.y, 0)
            x_end = min(self.x + w, self.source_background.shape[1])
            y_end = min(self.y + h, self.source_background.shape[0])

            src_x_start = 0 if self.x >= 0 else -self.x
            src_y_start = 0 if self.y >= 0 else -self.y
            src_x_end = w - (self.x + w - x_end)
            src_y_end = h - (self.y + h - y_end)

            # Update black background
            self.source_background[y_start:y_end, x_start:x_end] = self.temp_source[src_y_start:src_y_end, src_x_start:src_x_end]

            # Update target image with dimmed source overlay
            overlay = self.temp_source[src_y_start:src_y_end, src_x_start:src_x_end]
            if self.resized_target is not None:
                target_region = self.resized_target[y_start:y_end, x_start:x_end]
            else:
                target_region = self.temp_target[y_start:y_end, x_start:x_end]
            dimmed_overlay = cv2.addWeighted(target_region, 0.7, overlay, 0.3, 0)  # Adjust dimming as needed
            self.temp_target[y_start:y_end, x_start:x_end] = dimmed_overlay
            
            self.display_source_background()
            self.display_target_image()
            self.display_output_image()

    def display_target_image(self):
        if self.temp_target is not None:
            display_image = ImageTk.PhotoImage(Image.fromarray(self.temp_target))
            self.target_canvas.img = display_image
            self.target_canvas.create_image(0, 0, anchor="nw", image=display_image)

    def display_source_background(self):
        if self.source_background is not None:
            display_image = ImageTk.PhotoImage(Image.fromarray(self.source_background))
            self.background_canvas.img = display_image
            self.background_canvas.create_image(0, 0, anchor="nw", image=display_image)

    def display_output_image(self):
        if self.output_image is not None:
            display_image = ImageTk.PhotoImage(Image.fromarray(self.output_image))
            self.output_canvas.img = display_image
            self.output_canvas.create_image(0, 0, anchor="nw", image=display_image)
            
    def apply_pyramid_blending(self):
        if self.target_image is not None and self.source_image is not None and self.mask is not None:
            # Ensure the mask matches the source image size
            if self.resized_target is not None:
                mask_resized = cv2.resize(self.mask, (self.temp_source.shape[1], self.resized_target.shape[0]))
                # Blend the images
                start = time.time()
                blended_image = blend_with_pyramids(self.resized_target, self.source_background, mask_resized)
                end = time.time()
            else:
                mask_resized = cv2.resize(self.mask, (self.temp_source.shape[1], self.temp_source.shape[0]))
                # Blend the images
                start = time.time()
                blended_image = blend_with_pyramids(self.target_image, self.source_background, mask_resized)
                end = time.time()
            
            duration = (end - start) * 1000
            
            # Display the blended result
            self.output_image = blended_image
            self.display_output_image()
            self.log_message("Blending Applied.")
            self.log_message(f"Blending runtime: {duration} ms")
        else:
            self.log_message("Please load target&source image and draw a mask first.")

    def mouse_action_start(self, event):
        if self.mode.get() == "draw":
            self.drawing = True
            self.start_x, self.start_y = event.x, event.y  # Record the starting point for shapes
        elif self.mode.get() == "drag":
            h, w = self.temp_source.shape[:2]
            if self.x <= event.x < self.x + w and self.y <= event.y < self.y + h:
                self.dragging = True

    def mouse_action(self, event):
        if self.drawing and self.mode.get() == "draw":
            if self.mask_type.get() == "freeform":
                # Draw freeform strokes on the mask
                cv2.circle(self.mask, (event.x, event.y), 10, 255, -1)
            elif self.mask_type.get() == "rectangle":
                # Draw a rectangle (dynamic during mouse drag)
                self.clear_mask()
                cv2.rectangle(self.mask, (self.start_x, self.start_y), (event.x, event.y), 255, -1)
            elif self.mask_type.get() == "ellipse":
                # Draw an ellipse (dynamic during mouse drag)
                self.clear_mask()
                center = ((self.start_x + event.x) // 2, (self.start_y + event.y) // 2)
                axes = (abs(self.start_x - event.x) // 2, abs(self.start_y - event.y) // 2)
                cv2.ellipse(self.mask, center, axes, 0, 0, 360, 255, -1)
            elif self.mask_type.get() == "erase":
                # Erase strokes on the mask by freeform
                cv2.circle(self.mask, (event.x, event.y), 10, 0, -1)
            
            self.update_masked_source()
            self.update_images()
            
        elif self.dragging and self.mode.get() == "drag":
            # Dragging mode for moving the source image
            self.x, self.y = event.x, event.y
            self.update_images()

    def mouse_action_stop(self, event):
        self.drawing = False
        self.dragging = False

    def save_outputs(self):
        if self.source_background is not None and self.mask is not None and self.output_image is not None:
            cv2.imwrite("source_background.png", cv2.cvtColor(self.source_background, cv2.COLOR_RGB2BGR))
            cv2.imwrite("output_image.png", cv2.cvtColor(self.output_image, cv2.COLOR_RGB2BGR))
            self.log_message("Outputs saved: 'source_background.png', and 'output_image.png'")
        else:
            self.log_message("Please load target&source image and draw a mask first.")

    def save_clipped_image(self):
        if self.mask is not None and self.source_image is not None:
            clipped_image = np.zeros_like(self.source_background)
            for i in range(3):  # Apply mask to each channel
                clipped_image[:, :, i] = self.source_background[:, :, i] * (self.mask / 255)
            cv2.imwrite("clipped_image.png", cv2.cvtColor(clipped_image, cv2.COLOR_RGB2BGR))
            cv2.imwrite("mask.png", self.mask)
            self.log_message("Clipped images saved: 'clipped_image.png', 'mask.png'.")
        else:
            self.log_message("Please load source image and draw a mask first.")
    
    def clear_mask(self):
        if self.mask is not None:
            # Reset the mask to an empty state
            self.mask = np.zeros_like(self.mask, dtype=np.uint8)
            
            self.update_masked_source()
            self.update_images()
    
    def mask_cleared(self):
        self.clear_mask()
        self.log_message("Mask Cleared.")
            
    def clear_images(self):
        self.target_image = None
        self.source_image = None
        self.black_background = None
        self.source_background = None
        self.temp_target = None
        self.mask = None
        self.masked_source = None
        self.output_image = None
        self.target_canvas.delete("all")
        self.background_canvas.delete("all")
        self.output_canvas.delete("all")
        self.log_message("All image cleared.")
        
    def log_message(self, message):
        self.status_text.config(state="normal")
        self.status_text.insert("end", message + "\n")
        self.status_text.see("end")
        self.status_text.config(state="disabled")

if __name__ == "__main__":
    root = Tk()
    app = Mask_Blending_GUI(root)
    root.mainloop()
